In [3]:
pip install langchain-experimental

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for langchain-experimental from https://files.pythonhosted.org/packages/4d/4d/81725def89f72ac878be289929e8870fd5919744a8b603ad724f0263d61e/langchain_experimental-0.0.57-py3-none-any.whl.metadata
  Obtaining dependency information for langchain<0.2.0,>=0.1.15 from https://files.pythonhosted.org/packages/ed/3e/93045d37eba24e0b5eb05312e30cd9e12805ea5f1ae9ba51ec8a7d2f5372/langchain-0.1.16-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.2.0,>=0.1.41 from https://files.pythonhosted.org/packages/1d/93/0f6969c7d8a9cd220e600d10ccc16a8e5f1f27e55310ae077d8c3bc8d354/langchain_core-0.1.42-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community<0.1,>=0.0.32 from https://files.pythonhosted.org/packages/6d/9e/2af18bcaebd47995ec5a4e3eeb7c6cf8e6a99c637a7cafe09bb98fc0f6f5/langchain_community-0.0.32-py3-none-any.whl.metadata
   ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_openai import ChatOpenAI

In [22]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
llm= ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [4]:
class MedicalBilling(BaseModel):
    patient_id: int
    patient_name: str
    diagnosis_code: str
    procedure_code: str
    total_charge: float
    insurance_claim_amount: float

In [5]:
examples = [
    {
        "example": """Patient ID: 123456, Patient Name: John Doe, Diagnosis Code: 
        J20.9, Procedure Code: 99203, Total Charge: $500, Insurance Claim Amount: $350"""
    },
    {
        "example": """Patient ID: 789012, Patient Name: Johnson Smith, Diagnosis 
        Code: M54.5, Procedure Code: 99213, Total Charge: $150, Insurance Claim Amount: $120"""
    },
    {
        "example": """Patient ID: 345678, Patient Name: Emily Stone, Diagnosis Code: 
        E11.9, Procedure Code: 99214, Total Charge: $300, Insurance Claim Amount: $250"""
    },
]

In [6]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

In [18]:
synthetic_data_generator = create_openai_data_generator(
    output_schema=MedicalBilling,
    llm=ChatOpenAI(
        temperature=1,
        base_url="http://localhost:1234/v1", api_key="lm-studio",
    ),  # You'll need to replace with your actual Language Model instance
    prompt=prompt_template,
)

In [ ]:
synthetic_results = synthetic_data_generator.generate(
    subject="medical_billing",
    extra="the name must be chosen at random. Make it something you wouldn't normally choose.",
    runs=1,
)